In [78]:
from pathlib import Path
import pyrosetta
import pyrosetta_help as ph
from types import ModuleType
from collections import Counter
prc: ModuleType = pyrosetta.rosetta.core
prp: ModuleType = pyrosetta.rosetta.protocols
prcc: ModuleType = pyrosetta.rosetta.core.conformation
pr_scoring: ModuleType = pyrosetta.rosetta.core.scoring
pr_options: ModuleType = pyrosetta.rosetta.basic.options


logger = ph.configure_logger()
pyrosetta.distributed.maybe_init(extra_options=ph.make_option_string(no_optH=False,
                                                                     ex1=None,
                                                                     ex2=None,
                                                                     # mute='all',
                                                                     ignore_unrecognized_res=True,
                                                                     load_PDB_components=False,
                                                                     ignore_waters=True)
                                 )

acession = '4FVD'
pose: pyrosetta.Pose = pyrosetta.toolbox.rcsb.pose_from_rcsb(acession)
#blue = Blueprinter.from_pose(pose)

In [79]:
chains = pose.split_by_chain()
print('chains', prc.pose.conf2pdb_chain(pose))
print('sequence', *map(pyrosetta.Pose.sequence, chains))

chains map_unsigned_long_char{1: A, 2: C}
sequence SGAIYVGNFRVVNRHLATHNDWANLVWEDSSRDLLVSSTTAQGCDTIARCNCQTGVYYCNSRRKHYPVSFSKPSLIYVEASEYYPARYQSHLMLAQGHSEPGDAGGILRCQHGVVGIVSTGGNGLVGFADVRDLLWLD TLGK


In [81]:
# grafting is for within not at end:
# flipped = prp.grafting.insert_pose_into_pose(scaffold_pose=chains[2], 
#                                                insert_pose=chains[1],
#                                                insert_point=len(flipped.residues) -1,
#                                                copy_pdbinfo=True)

flipped = chains[2].clone()
prc.pose.append_pose_to_pose(pose1=flipped, pose2=chains[1], new_chain=False)

chains = flipped.split_by_chain()
print('chains', prc.pose.conf2pdb_chain(flipped))
print('sequence', *map(pyrosetta.Pose.sequence, chains))

chains map_unsigned_long_char{1: C}
sequence TLGKSGAIYVGNFRVVNRHLATHNDWANLVWEDSSRDLLVSSTTAQGCDTIARCNCQTGVYYCNSRRKHYPVSFSKPSLIYVEASEYYPARYQSHLMLAQGHSEPGDAGGILRCQHGVVGIVSTGGNGLVGFADVRDLLWLD


In [82]:
blue = ph.Blueprinter.from_pose(flipped)
blue.pick_native(3)
blue.pick_native(4)
blue.pick_native(5)
blue.pick_native(6)
blue.pick_native(7)
blue.pick_native(8)
blue.pick_native(9)
blue.insert(5,'PIKAA F')
blue.insert(5,'PIKAA G')
blue.insert(5,'PIKAA Q')
blue.insert(5,'PIKAA Q')
blue.set('mut.blu')

In [ ]:
pr_options.set_boolean_option('remodel:quick_and_dirty', False)
pr_options.set_string_option('remodel:generic_aa', 'G')

rm = prp.forge.remodel.RemodelMover()
rm.register_options()
rm.dr_cycles(15) # default 3
rm.max_linear_chainbreak(0.2) # default 0.07
rm.redesign_loop_neighborhood(False)
rm.apply(flipped)

AA for build: GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG


In [ ]:
flipped.dump_pdb(f'{accession}_joined.pdb')

In [ ]:
flipped.sequence()